In [2]:
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
from pymongo import MongoClient
from datetime import datetime, timedelta
import os
from config import Config

# Initialize MongoDB client
client = MongoClient(Config.MONGO_URI)
db = client['RemindMe-test']
collection = db['reminders']

In [62]:
start_date="2025-01-01"
end_date="2025-05-16"

# Fetch data from MongoDB and return DataFrame
def get_reminders_data(collection, start_date, end_date):
    """
    Busca en Mongo los datos de uso de la funcionalidad de remind me
    Retorna todos los documentos de la base en el rango dado con los campos
    El dataframe de salida tiene las columnas
    user_id: nro de teléfono del usuario
    date_time (str de la forma yyyy-mm-dd): fecha de creación del recordatorio
    sentAt (str de la forma yyyy-mm-dd): fecha de envío del recordatorio, si no se envió None
    status (str): sent si el recordatorio se envió, not_sent si no se envió
    """
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d') + timedelta(days=1)

    match_stage = {"$match": {"date_time": 
                            {"$gte": start.strftime('%Y-%m-%dT00:00:00.000-04:00'),
                            "$lt": end.strftime('%Y-%m-%dT00:00:00.000-04:00')}}}
    
    # Un solo proyecto que incluya todos los campos necesarios
    project_stage = {
        "$project": {
            "user_id": 1,
            "date_time": { "$substr": ["$date_time", 0, 10] },
            "sentAt": { "$ifNull": [{ "$substr": ["$sentAt", 0, 10] }, None] },
            "status": { "$ifNull": ["$status", "not_sent"] },
            "_id": 0
        }
    }
    pipeline = [match_stage, project_stage]

    data = list(collection.aggregate(pipeline))
    return pd.DataFrame(data)

# Metric functions
def get_daily_users(df):
    """Users creating reminders per day."""
    if df.empty:
        return pd.DataFrame({"date": [], "users": []})
    daily_users = df.groupby('date_time')['user_id'].nunique().reset_index(name='count')
    return daily_users

def get_monthly_users(df):
    """Users creating reminders per month."""
    if df.empty:
        return pd.DataFrame({"date": [], "users": []})
    df['mes'] = pd.to_datetime(df['date_time']).dt.to_period('M').astype(str)
    df_month = df.groupby('mes')['user_id'].nunique().reset_index(name='count')
    return df_month

def get_daily_reminds_created(df):
    """Reminders created per day."""
    if df.empty:
        return pd.DataFrame({"date": [], "created": [], "sent": []})
    # Group by date_created and count
    daily_counts = df.groupby('date_time').size().reset_index(name='count')
    return daily_counts.sort_values('date_time').reset_index(drop=True)

def get_monthly_reminds_created(df):
    """Reminders created per month."""
    if df.empty:
        return pd.DataFrame({"month": [], "created": [], "sent": []})
    # Convert date_time to datetime and extract month
    df['month'] = pd.to_datetime(df['date_time'], errors='coerce').dt.strftime('%Y-%m')
    # Group by month and count
    monthly_counts = df.groupby('month').size().reset_index(name='count')
    return monthly_counts

def get_daily_reminds_sent(df):
    sent_date = df[df['status'] == "sent"]
    daily_sent = sent_date.groupby('date_time').size().reset_index(name='count')
    return daily_sent

def get_monthly_reminders_sent(df):
    """Return DataFrame with month and count of status == 'sent' per month."""
    # Filter for status == 'sent'
    sent_date = df[df['status'] == 'sent']
    # Convert date_time to datetime and extract month
    sent_date['month'] = pd.to_datetime(sent_date['date_time'], errors='coerce').dt.strftime('%Y-%m')
    # Group by month and count
    monthly_sent = sent_date.groupby('month').size().reset_index(name='count')
    return monthly_sent

In [65]:
data = get_reminders_data(collection, start_date, end_date)
daily_users = get_daily_users(data)
monthly_users = get_monthly_users(data)
daily_reminds_created = get_daily_reminds_created(data)
monthly_reminds_created = get_monthly_reminds_created(data)
daily_reminds_sent = get_daily_reminds_sent(data)
monthly_reminds_sent = get_monthly_reminders_sent(data)

C:\Users\Felo\AppData\Local\Temp\ipykernel_16268\1121443316.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sent_date['month'] = pd.to_datetime(sent_date['date_time'], errors='coerce').dt.strftime('%Y-%m')


In [52]:
# Totals
total_users = data['user_id'].nunique(),
total_reminds_created = data['date_time'].count(),
total_reminds_sent = data['status'].value_counts()['sent'],

for i in [total_users, total_reminds_created, total_reminds_sent]:
    print (i)


(10,)
(np.int64(184),)
(np.int64(117),)


In [57]:
# Stats users
average_daily_users = daily_users['count'].mean()
average_monthly_users = monthly_users['count'].mean()
per_user_reminds_created = total_reminds_created[0] / total_users
per_user_reminds_sent = total_reminds_sent[0] / total_users

In [58]:
for i in [average_daily_users, average_monthly_users, per_user_reminds_created, per_user_reminds_sent]:
    print(i)

4.6
10.0
[18.4]
[11.7]


In [59]:
# Daily stats
average_daily_reminds_created = daily_reminds_created['count'].mean()
average_daily_reminds_sent = daily_reminds_sent['count'].mean()
average_per_user_daily_reminds_created = average_daily_reminds_created / total_users
average_per_user_daily_reminds_sent = average_daily_reminds_sent / total_users

In [60]:
for i in [average_daily_reminds_created, average_daily_reminds_sent, average_per_user_daily_reminds_created,average_per_user_daily_reminds_sent]:
    print(i)

36.8
23.4
[3.68]
[2.34]


In [66]:
# Monthly stats
average_monthly_reminds_created = monthly_reminds_created['count'].mean()
average_monthly_reminds_sent = monthly_reminds_sent['count'].mean()
average_per_user_monthly_reminds_created = average_monthly_reminds_created /total_users
average_per_user_monthly_reminds_sent = average_monthly_reminds_sent / total_users

In [67]:
for i in [average_monthly_reminds_created, average_monthly_reminds_sent,average_per_user_monthly_reminds_created, average_per_user_monthly_reminds_sent ]:
    print(i)

184.0
117.0
[18.4]
[11.7]
